# Reinforcement learning for Time series forecasting

In [1]:
from utilities.std_imports import *
from utilities.tf_imports import *

import tensorflow.keras.optimizers as tko
import math
import random
import sys
from collections import deque

In [2]:
class Agent:
    
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size # normalized previous days
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = load_model(model_name) if is_eval else self._model()
    
    def _model(self):
        model = tkm.Sequential()
        model.add(tkl.Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(tkl.Dense(units=32, activation="relu"))
        model.add(tkl.Dense(units=8, activation="relu"))
        model.add(tkl.Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=tko.Adam(lr=0.001))
        return model
    
    def act(self, state):
        if not self.is_eval and random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        options = self.model.predict(state)
        return np.argmax(options[0])
    
    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])
        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [3]:
def formatPrice(n):
    return("-$" if n<0 else "$")+"{0:.2f}".format(abs(n))

def getStockDataVec(key):
    vec = []
    lines = open(csv_path + 'time_series/' + key + ".csv","r").read().splitlines()
    for line in lines[1:]:
        #print(line)
        #print(float(line.split(",")[4]))
        vec.append(float(line.split(",")[4]))
        #print(vec)
    return vec 

def sigmoid(x):
    return 1/(1+math.exp(-x))

def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        res.append(sigmoid(block[i + 1] - block[i]))
    return np.array([res])

In [ ]:
stock_name = 'ACN'
window_size = 120
episode_count = 12
agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32

for e in range(episode_count + 1):
    print("\nEpisode " + str(e) + "/" + str(episode_count))
    state = getState(data, 0, window_size + 1)
    total_profit = 0
    agent.inventory = []
    for t in range(l):
        action = agent.act(state)
        # sit
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0
        if action == 1: # buy
            agent.inventory.append(data[t])
            print("Buy: " + formatPrice(data[t]))
        elif action == 2 and len(agent.inventory) > 0: # sell
            bought_price = window_size_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print("--------------------------------")
            print("Total Profit: " + formatPrice(total_profit))
            print("--------------------------------")
        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)
    if e % 10 == 0:
        agent.model.save(str(e))


Episode 0/12
Buy: $115.59
Sell: $115.10 | Profit: -$0.49
Buy: $114.10
Sell: $114.61 | Profit: $0.51
Buy: $112.92
Sell: $114.40 | Profit: $1.48
Buy: $113.55
Buy: $114.08
Buy: $115.55
Sell: $115.47 | Profit: $1.92
Sell: $115.44 | Profit: $1.36
Sell: $116.22 | Profit: $0.67
Buy: $116.52
Sell: $116.03 | Profit: -$0.49
Buy: $116.47
Buy: $119.12
Buy: $118.55
Sell: $118.41 | Profit: $1.94
Buy: $118.94
Buy: $119.01
Sell: $118.90 | Profit: -$0.22
Buy: $119.60
Buy: $119.65
Sell: $119.32 | Profit: $0.77
Sell: $119.44 | Profit: $0.50
Sell: $118.27 | Profit: -$0.74
Sell: $117.08 | Profit: -$2.52
Buy: $117.45
Sell: $117.57 | Profit: -$2.08
Buy: $119.05
Buy: $119.43
Sell: $118.91 | Profit: $1.46
Sell: $111.45 | Profit: -$7.60
Sell: $109.29 | Profit: -$10.14
Buy: $111.76
Buy: $113.29
Buy: $113.52
Buy: $115.11
Buy: $115.46
Buy: $116.83
Buy: $115.09
Sell: $113.37 | Profit: $1.61
Buy: $115.45
Sell: $114.26 | Profit: $0.97
Buy: $112.08
Sell: $112.81 | Profit: -$0.71
Buy: $112.83
Sell: $113.61 | Profit: -

### Credits & Links

https://www.analyticsvidhya.com/blog/2020/10/reinforcement-learning-stock-price-prediction/